In [27]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    # Abre el archivo PDF en modo lectura binaria
    with open(pdf_path, 'rb') as file:
        # Crea un lector de PDF
        pdf_reader = PyPDF2.PdfReader(file)
        
        # Verifica el número de páginas en el PDF
        num_pages = len(pdf_reader.pages)
        print(f"El PDF tiene {num_pages} páginas.")

        # Itera sobre cada página y extrae el texto
        text = ""
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()

    # Convierte el texto extraído a minúsculas
    return text.lower()

# Ruta al archivo PDF
pdf_path = r"C:\Users\Nahuel\Documents\GitHub\Proyecto-Facturas\factura_train\factura_0.pdf"

# Extrae el texto del PDF y convierte a minúsculas
pdf_text = extract_text_from_pdf(pdf_path)

# Muestra el texto extraído en minúsculas
print(pdf_text)


El PDF tiene 3 páginas.
datos  de la factura  
nº factura:  sv5043664894  
referencia:  591313314378/6522  
fecha  emisión  factura:  27/09/2018  
periodo  de facturación:  del 26/08/2018  a 25/09/2018  (30 días)  
fecha  de cargo:  30 de septiembre de 2018  
............................................................................................................................. .......................................  
iberdesa comercializadora sociedad limitada . 
cif b90393497 . 
crt.sevilla -madrid, km 524, camino de la pastora s/n  41410  - carmona  conrado daniel iglesias   
calle la solana  
22394  la fueva  
huesca  
forma  de pago:  domiciliada  potencia  
energía  
descuentos  
otros  
impuestos  
igic  reducido  16,80  € 
0,00 € 
-x,xx € 
0,80 € 
0,86 € 
1,24 € fecha  de cargo:  30 de septiembre de 2018  
iban:  es31873903585096496*****  
cod.mandato:  e88656958265427260292185691  
( 7%) 
versión:  4779  igic  normal  ( 10%) 0,08 € 
......................................

In [61]:
import fitz  # PyMuPDF
import re

def extraer_informacion_factura(ruta_pdf):
    try:
        documento = fitz.open(ruta_pdf)
        texto_completo = ""

        for pagina_num in range(len(documento)):
            pagina = documento.load_page(pagina_num)
            texto = pagina.get_text("text")
            texto_completo += texto + "\n"
        
        # Convierte el texto extraído a minúsculas
        texto_completo = texto_completo.lower()

        datos_factura = {}

        # Expresiones regulares para capturar datos específicos
        patrones = {
            "nombre_cliente": r"(?:titular del contrato|nombre|titular):\s*(.*)", #funciona en los 4 pdf
            "dni_cliente": r"nif:\s*(\w+)", #funciona en los 4 pdf
            "calle_cliente": r"(?:dirección de suministro|dirección suministro):\s*(.*?)(?=\s+\d|,|\n)", #funciona en los 4 pdf
            "cp_cliente": r"(?:dirección de suministro|dirección suministro):\s*(.*?)",
            "población_cliente": r"dirección de suministro:.*?,\s*\d+\s+(.*?),",
            "provincia_cliente": r"dirección de suministro:.*?,\s*\d+\s+.*?,\s*(\w+)",
            # "nombre_comercializadora": r"(iberdersa comercializadora sociedad limitada)",
            "cif_comercializadora": r"cif\s*(\w+)", #funciona en los 4 pdf
            "dirección_comercializadora": r"crt\..*?,\s*(km\s*\d+,\s*camino\s*de\s*la\s*pastora\s*s/n)",
            "cp_comercializadora": r"(\d{5})\s*-\s*carmona",
            "población_comercializadora": r"\d{5}\s*-\s*(carmona)",
            "provincia_comercializadora": r"carmona,\s*(sevilla)",
            "número_factura": r"nº factura:\s*(\w+)", #funciona en los 4 pdf
            "inicio_periodo": r"periodo de facturación:\s*del\s*(\d{2}/\d{2}/\d{4})",
            "fin_periodo": r"periodo de facturación:.*a\s*(\d{2}/\d{2}/\d{4})",
            "importe_factura": r"total importe factura\s*(\d+,\d+)",
            "fecha_cargo": r"fecha de cargo:\s*(\d{2}/\d{2}/\d{4})",
            "consumo_periodo": r"consumo en el periodo:\s*(\d+)\s*kwh",
            "potencia_contratada": r"potencia contratada:\s*(\d+,\d+)" #funciona en los 4 pdf
        }

        # Divide el texto en líneas para buscar el nombre de la comercializadora
        lineas = texto_completo.split('\n')
        nombre_comercializadora = None
        for i, linea in enumerate(lineas):
            if "cif" in linea:
                if i > 0:
                    nombre_comercializadora = lineas[i - 1].strip()
                break
        
        # Añadir el nombre de la comercializadora a los datos de la factura
        if nombre_comercializadora:
            datos_factura["nombre_comercializadora"] = nombre_comercializadora

        for clave, patron in patrones.items():
            resultado = re.search(patron, texto_completo, re.MULTILINE)
            if resultado:
                datos_factura[clave] = resultado.group(1).strip()
                
                # # Si se captura la calle, buscar el código postal en esa línea
                # if clave == "calle_cliente":
                #     linea_con_calle = resultado.group(0)
                #     cp_resultado = re.search(patrones["cp_cliente"], linea_con_calle)
                #     if cp_resultado:
                #         datos_factura["cp_cliente"] = cp_resultado.group(0).strip()

        # Convertir fechas y otros datos específicos (si es necesario)
        # datos_factura["inicio_periodo"] = re.sub(r"/", ".", datos_factura.get("inicio_periodo", ""))
        # datos_factura["fin_periodo"] = re.sub(r"/", ".", datos_factura.get("fin_periodo", ""))
        # datos_factura["fecha_cargo"] = re.sub(r"/", ".", datos_factura.get("fecha_cargo", ""))
        # datos_factura["consumo_periodo"] = int(datos_factura.get("consumo_periodo", "0"))

        return datos_factura

    except Exception as e:
        print(f"Error al procesar el PDF: {e}")
        return None

# Ruta al archivo PDF
ruta_pdf = r"C:\Users\Nahuel\Documents\GitHub\Proyecto-Facturas\factura_train\factura_10.pdf"

# Extraer y mostrar la información estructurada de la factura
informacion_factura = extraer_informacion_factura(ruta_pdf)
if informacion_factura:
    print(informacion_factura)
else:
    print("No se pudo extraer información del PDF.")


{'nombre_comercializadora': 'electrica del montsec sl', 'nombre_cliente': 'laureana peralta acosta', 'dni_cliente': '33040846n', 'calle_cliente': 'calle de sierra gorda', 'cp_cliente': '', 'cif_comercializadora': 'b25267220', 'número_factura': 'xv8868210822', 'importe_factura': '121,12', 'potencia_contratada': '2,95'}


In [51]:
import pandas as pd
#factura 0
informacion_factura

{'nombre_comercializadora': 'iberdesa comercializadora sociedad limitada.',
 'nombre_cliente': 'conrado daniel iglesias',
 'dni_cliente': '73635161v',
 'calle_cliente': 'calle la solana',
 'cp_cliente': '',
 'cif_comercializadora': 'b90393497',
 'dirección_comercializadora': 'km 524, camino de la pastora s/n',
 'cp_comercializadora': '41410',
 'población_comercializadora': 'carmona',
 'número_factura': 'sv5043664894',
 'inicio_periodo': '26/08/2018',
 'fin_periodo': '25/09/2018',
 'importe_factura': '19,78',
 'potencia_contratada': '4,80'}

In [53]:
import pandas as pd
#factura 1
informacion_factura

{'nombre_comercializadora': 'dese de alta gratis en http://enigma-technologies.com//mifactura',
 'nombre_cliente': 'leonarda jaramillo báez',
 'dni_cliente': '90237166r',
 'calle_cliente': 'camino de los pelendengues',
 'cp_cliente': '',
 'cif_comercializadora': 'b88446406',
 'número_factura': 'k9131055493',
 'potencia_contratada': '4,524'}

In [55]:
import pandas as pd
#factura 6
informacion_factura

{'nombre_comercializadora': 'comercialitzadora electrica de montoliu de lleida s.l.u',
 'nombre_cliente': 'dimitri segura lucas',
 'dni_cliente': '13428402j',
 'calle_cliente': 'calle santiago de rusiñol',
 'cp_cliente': '',
 'cif_comercializadora': 'b25813767',
 'número_factura': 'st6455016904',
 'importe_factura': '87,16',
 'potencia_contratada': '4,097'}

In [59]:
import pandas as pd
#factura 10
informacion_factura

{'nombre_comercializadora': 'electrica del montsec sl',
 'nombre_cliente': 'laureana peralta acosta',
 'dni_cliente': '33040846n',
 'calle_cliente': 'calle de sierra gorda',
 'cp_cliente': '',
 'cif_comercializadora': 'b25267220',
 'número_factura': 'xv8868210822',
 'importe_factura': '121,12',
 'potencia_contratada': '2,95'}

In [11]:
import re
import PyPDF2

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)
        text = ""
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

def extract_info(text):
    # Patrones para extraer información específica
    nombre_cliente_pattern = re.compile(r'Titular\s+del\s+contrato:\s+([^\n]+)')
    dni_cliente_pattern = re.compile(r'NIF:\s+(\S+)')
    calle_cliente_pattern = re.compile(r'Dirección\s+de\s+suministro:\s+([^\n,]+),')
    cp_cliente_pattern = re.compile(r'\n(\d{5})\s')
    poblacion_cliente_pattern = re.compile(r'Dirección\s+de\s+suministro:.*?,\s+\d{5}\s+([^,\n]+)')
    provincia_cliente_pattern = re.compile(r'Dirección\s+de\s+suministro:.*?,\s+\d{5}\s+[^,\n]+,\s+([^\n]+)')
    nombre_comercializadora_pattern = re.compile(r'IBERDESA COMERCIALIZADORA SOCIEDAD LIMITADA')
    cif_comercializadora_pattern = re.compile(r'CIF\s+(\S+)')
    direccion_comercializadora_pattern = re.compile(r'CRT\.SEVILLA-MADRID, KM 524, CAMINO DE LA PASTORA S/N')
    cp_comercializadora_pattern = re.compile(r'41410')
    poblacion_comercializadora_pattern = re.compile(r'41410\s+-\s+([^,\n]+)')
    provincia_comercializadora_pattern = re.compile(r'41410\s+-\s+[^,\n]+,\s+([^\n]+)')
    numero_factura_pattern = re.compile(r'Nº factura:\s+(\S+)')
    inicio_periodo_pattern = re.compile(r'Periodo\s+de\s+Facturación:\s+del\s+(\d{2}/\d{2}/\d{4})')
    fin_periodo_pattern = re.compile(r'Periodo\s+de\s+Facturación:\s+del\s+\d{2}/\d{2}/\d{4}\s+a\s+(\d{2}/\d{2}/\d{4})')
    importe_factura_pattern = re.compile(r'TOTAL\s+IMPORTE\s+FACTURA\s+(\d+,\d+)\s+€')
    fecha_cargo_pattern = re.compile(r'Fecha\s+de\s+cargo:\s+(\d{2}\s+de\s+\w+\s+de\s+\d{4})')
    consumo_periodo_pattern = re.compile(r'Consumo\s+en\s+el\s+periodo\s+valle\s+(\d+)')
    potencia_contratada_pattern = re.compile(r'Potencia\s+contratada:\s+(\d+,\d+)')

    # Extrae la información usando los patrones
    info = {
        "nombre_cliente": nombre_cliente_pattern.search(text).group(1) if nombre_cliente_pattern.search(text) else None,
        "dni_cliente": dni_cliente_pattern.search(text).group(1) if dni_cliente_pattern.search(text) else None,
        "calle_cliente": calle_cliente_pattern.search(text).group(1) if calle_cliente_pattern.search(text) else None,
        "cp_cliente": cp_cliente_pattern.search(text).group(1) if cp_cliente_pattern.search(text) else None,
        "población_cliente": poblacion_cliente_pattern.search(text).group(1) if poblacion_cliente_pattern.search(text) else None,
        "provincia_cliente": provincia_cliente_pattern.search(text).group(1) if provincia_cliente_pattern.search(text) else None,
        "nombre_comercializadora": "IBERDESA COMERCIALIZADORA SOCIEDAD LIMITADA",
        "cif_comercializadora": cif_comercializadora_pattern.search(text).group(1) if cif_comercializadora_pattern.search(text) else None,
        "dirección_comercializadora": "CRT.SEVILLA-MADRID, KM 524, CAMINO DE LA PASTORA S/N",
        "cp_comercializadora": cp_comercializadora_pattern.search(text).group(0) if cp_comercializadora_pattern.search(text) else None,
        "población_comercializadora": poblacion_comercializadora_pattern.search(text).group(1) if poblacion_comercializadora_pattern.search(text) else None,
        "provincia_comercializadora": provincia_comercializadora_pattern.search(text).group(1) if provincia_comercializadora_pattern.search(text) else None,
        "número_factura": numero_factura_pattern.search(text).group(1) if numero_factura_pattern.search(text) else None,
        "inicio_periodo": inicio_periodo_pattern.search(text).group(1).replace("/", ".") if inicio_periodo_pattern.search(text) else None,
        "fin_periodo": fin_periodo_pattern.search(text).group(1).replace("/", ".") if fin_periodo_pattern.search(text) else None,
        "importe_factura": importe_factura_pattern.search(text).group(1).replace(",", ".") if importe_factura_pattern.search(text) else None,
        "fecha_cargo": fecha_cargo_pattern.search(text).group(1) if fecha_cargo_pattern.search(text) else None,
        "consumo_periodo": int(consumo_periodo_pattern.search(text).group(1)) if consumo_periodo_pattern.search(text) else None,
        "potencia_contratada": potencia_contratada_pattern.search(text).group(1).replace(",", ".") if potencia_contratada_pattern.search(text) else None
    }

    return info

# Ruta al archivo PDF
pdf_path = r"C:\Users\Nahuel\Documents\GitHub\Proyecto-Facturas\factura_train\factura_0.pdf"

# Extrae el texto del PDF
pdf_text = extract_text_from_pdf(pdf_path)

# Extrae la información específica
info = extract_info(pdf_text)

# Muestra la información extraída
print(info)


{'nombre_cliente': 'Conrado Daniel Iglesias  ', 'dni_cliente': '73635161V', 'calle_cliente': 'Calle la Solana', 'cp_cliente': '22394', 'población_cliente': None, 'provincia_cliente': None, 'nombre_comercializadora': 'IBERDESA COMERCIALIZADORA SOCIEDAD LIMITADA', 'cif_comercializadora': 'B90393497', 'dirección_comercializadora': 'CRT.SEVILLA-MADRID, KM 524, CAMINO DE LA PASTORA S/N', 'cp_comercializadora': '41410', 'población_comercializadora': 'CARMONA  Conrado Daniel Iglesias   ', 'provincia_comercializadora': None, 'número_factura': 'SV5043664894', 'inicio_periodo': '26.08.2018', 'fin_periodo': '25.09.2018', 'importe_factura': '19.78', 'fecha_cargo': '30 de septiembre de 2018', 'consumo_periodo': None, 'potencia_contratada': '4.799'}
